# Customizing Sequence Interval Classes

The core class in aligned-textgrid is the `SequenceInterval` from which new sequence interval classes can be subclassed. The `Word` and `Phone` classes are, themselves, subclasses of `SequenceInterval`.

In [1]:
from aligned_textgrid.aligned_textgrid import AlignedTextGrid
from aligned_textgrid.sequences.sequences import SequenceInterval
from aligned_textgrid.sequences.word_and_phone import Word, Phone
from aligned_textgrid.custom_classes import custom_classes

issubclass(Word, SequenceInterval)

True

The most important information every sequence interval class has is its `.superset_class` and `.subset_class`, which needs to be another (but different!) `SequenceInterval` subclass.

In [2]:
Word.subset_class

aligned_textgrid.sequences.word_and_phone.Phone

In [3]:
Phone.superset_class

aligned_textgrid.sequences.word_and_phone.Word

In [4]:
Word.subset_class.subset_class

aligned_textgrid.sequences.sequences.Bottom

## Creating custom classes with `custom_classes()`

If you don't want or need any additional bells and whistles beyond [what `SequenceInterval` already provides](/reference/aligned_textgrid/sequences/sequences/#aligned_textgrid.sequences.sequences.SequenceInterval), you can just use the `custom_classes()` function. For example, if you wanted your own interval classes for a Word, Syllable, and Phone:

In [5]:
MyWord, Syllable, MyPhone = custom_classes(
    class_list=["MyWord", "Syllable", "Phone"]
)

In [6]:
[MyWord.subset_class, Syllable.superset_class]

[aligned_textgrid.custom_classes.Syllable,
 aligned_textgrid.custom_classes.MyWord]

The reciprocal `.subset_class` and `.superset_class` relationships will be set from left to right for the class names provided to `class_list`.

### Using `custom_classes()` when creating an `AlignedTextGrid`

The `custom_classes()` returns a list of sequence interval classes, which is also a requried input when creating a new `AlignedTextGrid` object, so you you could create the new sequence interval classes on the fly.

Let's say you began with [this textgrid](../resources/spritely.TextGrid) representing the syllable structure of a word:

![syllable structure textgrid](../resources/spritely.png)

You could create an `AlignedTextGrid` object like so:

In [7]:
spritely = AlignedTextGrid(
    textgrid_path = "../resources/spritely.TextGrid",
    entry_classes = custom_classes(
        class_list = ["PrWord", "Foot", "Syl", "OnsetRime", "SylPart", "Phone"]
    )
)

We can see that the textgrid was successfully parsed by printing `spritely`.

In [8]:
print(spritely)

AlignedTextGrid with 1 groups, each with [6] tiers. [['PrWord', 'Foot', 'Syl', 'OnsetRime', 'SylPart', 'Phone']]


Within the first tier group, the tier with index `4` is the `SylPart` tier. Printing it, we can see that its superset class has been set to `OnsetRime` and its subset class has been set to `Phone`.

In [9]:
print(spritely[0][4])

Sequence tier of SylPart; .superset_class: OnsetRime; .subset_class: Phone


We can look at the labels for this tier:

In [10]:
print(spritely[0][4].labels)

['', 'Onset', 'Nucleus', 'Coda', 'Onset', 'Nucleus', '']


And we can print out some information about the index `1` interval.

In [11]:
print(spritely[0][4][1].label)
print(spritely[0][4][1].sub_labels)

Onset
['S', 'P', 'R']


### When the tier order != the hierarchy order

It's possible that you'll be working with textgrids where the largest superset is not the topmost tier. For example, here is [the spritely textgrid reversed](../resources/spritely_rev.png).

![spritely reversed](../resources/spritely_rev.png)

The `custom_classes()` function still needs the hierarchy to be arranged from left to right, but you can also specify a `return_order`, either by numeric index or by name, that the new classes get returned.

In [12]:
custom_classes(
    class_list = ["PrWord", "Foot", "Syl", "OnsetRime", "SylPart", "Phone"],
    return_order=[5, 4, 3, 2, 1, 0]
)
# -or-
# custom_classes(
#     class_list = ["PrWord", "Foot", "Syl", "OnsetRime", "SylPart", "Phone"],
#     return_order=["Phone", "SylPart", "OnsetRime", "Syl", "Foot", "PrWord"]
# )

[aligned_textgrid.custom_classes.Phone,
 aligned_textgrid.custom_classes.SylPart,
 aligned_textgrid.custom_classes.OnsetRime,
 aligned_textgrid.custom_classes.Syl,
 aligned_textgrid.custom_classes.Foot,
 aligned_textgrid.custom_classes.PrWord]

The order of these classes in the output list may have `Phone` first, but the topmost class in the hierarchy is still `PrWord`

In [13]:
spritely_rev = AlignedTextGrid(
    textgrid_path= "../resources/spritely_rev.TextGrid",
    entry_classes = custom_classes(
        class_list = ["PrWord", "Foot", "Syl", "OnsetRime", "SylPart", "Phone"],
        return_order=[5, 4, 3, 2, 1, 0]        
    )
)
print(spritely_rev)

AlignedTextGrid with 1 groups, each with [6] tiers. [['Phone', 'SylPart', 'OnsetRime', 'Syl', 'Foot', 'PrWord']]


In [15]:
spritely_rev[0][0]

Sequence tier of PrWord; .superset_class: Top; .subset_class: Foot

In [16]:
spritely_rev[0].tier_list

[Sequence tier of PrWord; .superset_class: Top; .subset_class: Foot,
 Sequence tier of Foot; .superset_class: PrWord; .subset_class: Syl,
 Sequence tier of Syl; .superset_class: Foot; .subset_class: OnsetRime,
 Sequence tier of OnsetRime; .superset_class: Syl; .subset_class: SylPart,
 Sequence tier of SylPart; .superset_class: OnsetRime; .subset_class: Phone,
 Sequence tier of Phone; .superset_class: SylPart; .subset_class: Bottom]